In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf 
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [7]:
## Load the trained model, Scaler pickle, onehot
model = load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)    


In [8]:
## Example input data
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France' ,
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure': 3,
    'Balance' : 60000,
    'NumOfProducts':2,
    'HasCrCard' : 1,
    'IsActiveMember':1,
    'EstimateSalary' : 50000

}

In [9]:
input_data_df = pd.DataFrame.from_dict([input_data])

In [10]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimateSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])

In [15]:
geo_encoder = onehot_encoder_geo.transform(input_data_df[['Geography']])

In [17]:
geo_encoder.toarray()

array([[1., 0., 0.]])

In [18]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [19]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_data_df = pd.concat([input_data_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [21]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimateSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [26]:
input_data_df.rename(columns={'EstimateSalary' : 'EstimatedSalary'},inplace=True)

In [27]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [28]:
input_data_df = scaler.transform(input_data_df)

In [29]:
input_data_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [30]:
prediction = model.predict(input_data_df)

1/1 [==============================] - 0s 149ms/step


In [31]:
prediction

array([[0.02735951]], dtype=float32)